## Parafac

Now the CANDECOMP/PARAFAC decomposition. The above tensor is too high a rank to reasonably decompose in this example, so we instead generate an example sparse tensor from a random sparse factorization and re-factor it.

In [1]:
import numpy as np
import sparse
import tensorly as tl

In [2]:
shape = (1000, 1001, 1002, 100)
rank = 5

starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors
starting_weights = sparse.ones(rank)

Now convert it to a tensor. It is very important to use `kruskal_to_tensor` from the sparse backend, as a fully dense version of the tensor would use several TB of memory.

In [3]:
from tensorly.contrib.sparse.kruskal_tensor import kruskal_to_tensor
tensor = kruskal_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002, 100)"
nnz,4737
Density,4.722822088091549e-08
Read-only,True
Size,185.0K
Storage ratio,0.0


As before, we can compare the actual spase used by the tensor vs. what it would require if it were dense.

In [4]:
tensor.nbytes / 1e9                # Actual memory usage in GB

0.00018948

In [5]:
np.prod(tensor.shape) * 8 / 1e9    # Memory usage if array was dense, in GB

802.4016

In [6]:
import time
%load_ext memory_profiler

Note that even though we started with a sparse tensor, the factors are dense. This is because we used the dense version of `parafac`. Since the factors are in general dense, even for a sparse tensor, this is generally preferred. 

Let's decompose the sparse tensor into a sparse Kruskal tensor:

In [7]:
from tensorly.contrib.sparse.decomposition import parafac as parafac_sparse

In [8]:
%%memit
start_time = time.time()
sparse_kruskal = parafac_sparse(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.5826844593958526
iteration 1, reconstruction error: 0.32583862278041187, decrease = 0.2568458366154407, unnormalized = 2.387513538370463
iteration 2, reconstruction error: 0.32499296587658, decrease = 0.0008456569038318706, unnormalized = 2.3813171664072916
iteration 3, reconstruction error: 0.32497680025263603, decrease = 1.616562394396448e-05, unnormalized = 2.3811987162196093
iteration 4, reconstruction error: 0.3249728063814774, decrease = 3.993871158625151e-06, unnormalized = 2.3811694519740745
iteration 5, reconstruction error: 0.3249713085966895, decrease = 1.4977847879182882e-06, unnormalized = 2.3811584772730763
iteration 6, reconstruction error: 0.3249706074264761, decrease = 7.011702133907782e-07, unnormalized = 2.381153339596754
iteration 7, reconstruction error: 0.3249702426319896, decrease = 3.647944865070585e-07, unnormalized = 2.38115066664237
iteration 8, reconstruction error: 0.324970039781819, decrease = 2.0285017060528432e-07, unnormalized = 2

Let's look at the result

In [9]:
sparse_kruskal

(weights, factors) : rank-5 KruskalTensor of shape (1000, 1001, 1002, 100) 

Because the `factors_sparse` are sparse, we can reconstruct them into a tensor without using too much memory. In general, this will not be the case, but it is for our toy example. Let's do this to look at the absolute error for the decomposition. 

In [10]:
tl.norm(tensor - kruskal_to_tensor(sparse_kruskal))

2.381147009853927

It is not actually necessary to compute this, as the same as the norm of the tensor times the reconstruction error that was printed by the algorithm (you can pass `return_errors=True` to `parafac()` to have the reconstruction errors be returned along with the factors). That is, $$\mathrm{reconstruction\ error} = \frac{\|\mathrm{tensor} - \mathrm{kruskal\_to\_tensor}(\mathrm{factors})\|_2}{\|\mathrm{tensor}\|_2}$$ (they won't be exactly the same due to numerical differences in how they are calculated).

In [11]:
tl.norm(tensor - kruskal_to_tensor(sparse_kruskal))/tl.norm(tensor)

0.3249697435676321

Let's look at one of the nonzero entries to see how close it is to the original tensor. The factors satisfy $$\sum_{r=0}^{R-1} {f_0}_r\circ {f_1}_r \circ {f_2}_r \circ {f_3}_r,$$ where $R$ is the rank (here 5), ${f_i}_r$ is the $r$-th column of the $i$-th factor of the decomposition, and $\circ$ is the vector outer product. Component-wise, this translates to a product of corresponding elements per component for each factor, summed over the columns.

In [12]:
tensor.coords

array([[  7,   7,   7, ..., 952, 952, 952],
       [ 10,  10,  10, ..., 955, 955, 955],
       [121, 260, 266, ..., 685, 810, 888],
       [ 10,  10,  10, ...,  91,  91,  91]])

In [13]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.00869538487621399

In [14]:
weights_sparse, factors_sparse = sparse_kruskal
sparse_val = np.sum(np.prod(sparse.stack([factors_sparse[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
sparse_val

0.0

In [16]:
np.abs(orig_val - sparse_val)

0.00869538487621399

The difference here is mostly due to random chance. The total reconstruction errors for the two runs of algorithm were roughly the same. In general, the error of the factorization will vary due to the randomness of the initial factors chosen by the algorithm.